In [24]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver

In [25]:
llm = ChatOllama(model='gemma3:4b')

In [26]:
class JokeState(TypedDict):
    
    topic: str 
    joke: str
    explanation: str

In [27]:
def generate_joke(state: JokeState):
    prompt = f"generate a joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content
    return {'joke': response}

In [28]:
def generate_explanation(state:JokeState):
    prompt = f"write an explanation of the joke- {state['joke']}"
    response = llm.invoke(prompt).content
    return {'explanation':response}

In [29]:
graph = StateGraph(JokeState)

#add nodes
graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

#add edges
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer  = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)


In [30]:
config1 = {'configurable': {'thread_id':'1'}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza break up with the bread? \n\nBecause it said, "You\'re always crusting me out!" 😂 \n\n---\n\nWould you like to hear another pizza joke?',
 'explanation': 'Okay, let’s break down why that joke is funny!\n\nThe joke plays on the double meaning of the word "crusting."\n\n*   **Literal Meaning:** Pizza is often topped with a crispy, golden-brown crust.\n*   **Figurative Meaning:** "Crusting me out" is a playful way of saying "you\'re always disappointing me" or “you’re always letting me down.”\n\nThe humor comes from the unexpected twist – a romantic breakup is described using a pizza-related term, making it silly and relatable.\n\n---\n\nYes, please! I’d love to hear another pizza joke. 😄'}

In [31]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza break up with the bread? \n\nBecause it said, "You\'re always crusting me out!" 😂 \n\n---\n\nWould you like to hear another pizza joke?', 'explanation': 'Okay, let’s break down why that joke is funny!\n\nThe joke plays on the double meaning of the word "crusting."\n\n*   **Literal Meaning:** Pizza is often topped with a crispy, golden-brown crust.\n*   **Figurative Meaning:** "Crusting me out" is a playful way of saying "you\'re always disappointing me" or “you’re always letting me down.”\n\nThe humor comes from the unexpected twist – a romantic breakup is described using a pizza-related term, making it silly and relatable.\n\n---\n\nYes, please! I’d love to hear another pizza joke. 😄'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0744f9-be5a-608b-8002-a5581052cd47'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T12:02:51.524109+00:00', pa

In [32]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza break up with the bread? \n\nBecause it said, "You\'re always crusting me out!" 😂 \n\n---\n\nWould you like to hear another pizza joke?', 'explanation': 'Okay, let’s break down why that joke is funny!\n\nThe joke plays on the double meaning of the word "crusting."\n\n*   **Literal Meaning:** Pizza is often topped with a crispy, golden-brown crust.\n*   **Figurative Meaning:** "Crusting me out" is a playful way of saying "you\'re always disappointing me" or “you’re always letting me down.”\n\nThe humor comes from the unexpected twist – a romantic breakup is described using a pizza-related term, making it silly and relatable.\n\n---\n\nYes, please! I’d love to hear another pizza joke. 😄'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0744f9-be5a-608b-8002-a5581052cd47'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-08T12:02:51.524109+00:00', p

In [33]:
config2 = {'configurable': {'thread_id': '2'}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti break up with the ravioli? \n\n... Because it said, "You\'re too cheesy!" 😄 \n\n---\n\nWould you like to hear another pasta joke?',
 'explanation': "Okay, let's break down the joke:\n\n“Why did the spaghetti break up with the ravioli? Because it said, ‘You’re too cheesy!’”\n\n**The humor lies in a play on words and a common association.**\n\n* **“Cheesy”** – Normally, “cheesy” describes a personality trait – someone overly sentimental, gooey, or a bit dramatic.\n* **Ravioli & Cheese:** Ravioli is almost always stuffed with cheese, and that cheese is a key part of what makes ravioli, well, ravioli! \n\nThe joke takes this literal association and uses “cheesy” in a way that’s unexpected and a little silly. It’s a lighthearted jab at the ravioli's abundance of cheese. 😄\n\n---\n\nAnd yes, absolutely! Would you like to hear another pasta joke?"}